In [1]:
import numpy as np # for random number distributions
import pandas as pd # for event_log data frame
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
import queue # add FIFO queue data structure
from functools import partial, wraps

import simpy # discrete event simulation environment


In [2]:
#set available resources
machine1 = 3

min_machine1_time = 5 #minutes (300 seconds)
mean_machine1_time = 10 #minutes (600 seconds)
max_machine1_time = 20 #minutes (1200 seconds)

mean_prep_time = 3 #minutes (900 seconds)

inventory1_queue_length = 20 #max amount of parts in queue before send to inventory

inv1 = 0 #prepared parts inventory for stage 1 machining

#available resources for line 2
machine2 = 2

min_machine2_time = 10 #minutes (600 seconds)
mean_machine2_time = 20 #minutes (1200 seconds)
max_machine2_time = 30 #minutes (1800 seconds)

inventory2_queue_length = 30 #max amount of parts in queue before send to inventory for stage 2

inv2 = 0

obtain_reproducible_results = True

#set length of simulation
simulation_days = 1
fixed_simulation_time = simulation_days*16*60*60 #16 operating hours from two 8 hour shifts

parameter_string_list = [str(simulation_days), 'days',
                        str(machine1),str(min_machine1_time),
                        str(mean_machine1_time),str(max_machine1_time),
                        str(mean_prep_time),str(inventory1_queue_length),
                        str(machine2),str(min_machine2_time),
                        str(mean_machine2_time),str(max_machine2_time),
                        str(inventory2_queue_length)]
separator = '-'
simulation_file_identifier = separator.join(parameter_string_list)

#initialize queue 1 length
#queue1 = 0

In [3]:
#function that generates random times for the 1st machining process
def random_machine1_time(min_machine1_time,mean_machine1_time,max_machine1_time):
    try_machine1_time = np.random.exponential(scale = mean_machine1_time)
    if (try_machine1_time < min_machine1_time):
        return(min_machine1_time)
    if (try_machine1_time > max_machine1_time):
        return(max_machine1_time)
    if (try_machine1_time >= min_machine1_time) and (try_machine1_time <= max_machine1_time):
        return(try_machine1_time)
    

In [35]:
#function that generates random times for the 2nd machining process
def random_machine2_time(min_machine2_time,mean_machine2_time,max_machine2_time):
    try_machine2_time = np.random.exponential(scale = mean_machine2_time)
    if (try_machine2_time < min_machine2_time):
        return(min_machine2_time)
    if (try_machine2_time > max_machine2_time):
        return(max_machine2_time)
    if (try_machine2_time >= min_machine2_time) and (try_machine2_time <= max_machine2_time):
        return(try_machine2_time)

In [5]:
#generates the prepped parts for the beginning of the process
def prep(env, caseid, caseid_queue1, caseid_inv1, event_log):
    #generates new parts prepared for machine stage 1
    caseid = 0
    while True:
        
        prep_time = round(60*np.random.exponential(scale = mean_prep_time))
        
        print("Next prepared part for Stage 1: ", env.now + prep_time)
        
        yield env.timeout(prep_time)
        
        caseid += 1
        time = env.now
        activity = 'prep_part'
        env.process(event_log_append(env, caseid, time, activity, event_log))
        yield env.timeout(0)
        
        #if-else statement determines if the prepared part should be stored in inventory or the machining queue
        if caseid_queue1.qsize() < inventory1_queue_length:
            caseid_queue1.put(caseid)
            print("Prepared part joins queue caseid =",caseid,'time =',env.now,'queue_length =',caseid_queue1.qsize())
            time = env.now
            activity = 'join_queue1'
            env.process(event_log_append(env, caseid, time, activity, event_log))
            env.process(machine1_process(env, caseid_queue1, event_log))
        else:
            caseid_inv1.put(caseid)
            print("Prepared part placed in Stage 1 Inventory. caseid=", caseid,'time =',env.now,'inv_size =',caseid_inv1.qsize())
            env.process(event_log_append(env, caseid, env.now, 'inv1', event_log))
            

In [53]:
#function for the 1st machining process
def machine1_process(env, caseid_queue, event_log):
    #must have available stage 1 machines
    with available_machine1.request() as req:
        yield req
        
        #if-else determines if we should be using inventory parts or queued parts
        #to follow first-in-first-out processes will need to modify this to fill the back of the queue when a spot opens up
        if caseid_inv1.qsize() > 0:
            queue1_length_on_entering_machining = caseid_inv1.qsize()
            caseid = caseid_inv1.get()
            print("Begin_stage1_machining for inventory parts, caseid =",caseid,'time = ',env.now,'inv1_length =', queue1_length_on_entering_machining)
            env.process(event_log_append(env, caseid, env.now, 'begin_stage1_machine', event_log))
            machine1_time = round(60*random_machine1_time(min_machine1_time,mean_machine1_time,max_machine1_time))
            yield env.timeout(machine1_time)
            queue1_length_on_leaving_machining = caseid_inv1.qsize()
            print("End_stage1_machining for inventory parts caseid=",caseid,'time = ',env.now,'queue1_length =',queue1_length_on_leaving_machining)
            env.process(event_log_append(env, caseid, env.now, 'end_stage1_machine', event_log))

            
        else:
            queue1_length_on_entering_machining = caseid_queue1.qsize()
            caseid = caseid_queue1.get()
            print("Begin_stage1_machining caseid =",caseid,'time = ',env.now,'queue1_length =',queue1_length_on_entering_machining)
            env.process(event_log_append(env, caseid, env.now, 'begin_stage1_machine', event_log))
            machine1_time = round(60*random_machine1_time(min_machine1_time,mean_machine1_time,max_machine1_time))
            yield env.timeout(machine1_time)
            queue1_length_on_leaving_machining = caseid_queue.qsize()
            print("End_stage1_machining caseid=",caseid,'time = ',env.now,'queue1_length =',queue1_length_on_leaving_machining)
            env.process(event_log_append(env, caseid, env.now, 'end_stage1_machine', event_log))

        #this puts the finished part into a transitional queue that we then pass to the next machining process
        caseid_transition.put(caseid)
        print("Transition to stage 2 caseid=",caseid,'time = ',env.now,'transition_queue_length =',caseid_transition.qsize())
        env.process(event_log_append(env, caseid, env.now, 'transition', event_log))
        env.process(machine2_process(env, caseid_transition, event_log))

In [51]:
#function for the 2nd machining process
def machine2_process(env, caseid_queue, event_log):
    #initializes the caseid with the transitional queue value that we pass from the 1st machine process
    caseid = caseid_transition.get()
    
    #if-else determines if we should place the machined part into inventory or the machining queue
    if caseid_queue2.qsize() < inventory2_queue_length:
        caseid_queue2.put(caseid)
        print("Machined part joins queue2 caseid =",caseid,'time =',env.now,'queue_length =',caseid_queue2.qsize())
        time = env.now
        activity = 'join_queue2'
        env.process(event_log_append(env, caseid, time, activity, event_log))
    else:
        caseid_inv2.put(caseid)
        print("Machined part placed in Stage 2 Inventory. caseid=", caseid,'time =',env.now,'inv_size =',caseid_inv2.qsize())
        env.process(event_log_append(env, caseid, env.now, 'inv2', event_log))
    
    with available_machine2.request() as req:
        yield req
      
        #if-else determines if we should use the inventory or the queue parts
        #will need to make adjustment that is noted in the first machining process about first-in-first-out
        if caseid_inv2.qsize() > 0:
            queue2_length_on_entering_machining = caseid_inv2.qsize()
            caseid = caseid_inv2.get()
            print("Begin_stage2_machining for inventory parts, caseid =",caseid,'time = ',env.now,'inv2_length =', queue2_length_on_entering_machining)
            env.process(event_log_append(env, caseid, env.now, 'begin_stage2_machine', event_log))
            machine2_time = round(60*random_machine2_time(min_machine2_time,mean_machine2_time,max_machine2_time))
            yield env.timeout(machine2_time)
            queue2_length_on_leaving_machining = caseid_inv2.qsize()
            print("End_stage2_machining for inventory parts caseid=",caseid,'time = ',env.now,'queue2_length =',queue2_length_on_leaving_machining)
            env.process(event_log_append(env, caseid, env.now, 'end_stage2_machine', event_log))
            
        else:
            queue2_length_on_entering_machining = caseid_queue2.qsize()
            caseid = caseid_queue2.get()
            print("Begin_stage2_machining caseid =",caseid,'time = ',env.now,'queue2_length =',queue2_length_on_entering_machining)
            env.process(event_log_append(env, caseid, env.now, 'begin_stage2_machine', event_log))
            machine2_time = round(60*random_machine2_time(min_machine2_time,mean_machine2_time,max_machine2_time))
            yield env.timeout(machine2_time)
            queue2_length_on_leaving_machining = caseid_queue.qsize()
            print("End_stage2_machining caseid=",caseid,'time = ',env.now,'queue2_length =',queue2_length_on_leaving_machining)
            env.process(event_log_append(env, caseid, env.now, 'end_stage2_machine', event_log))
        



In [28]:
def trace(env, callback):
     """Replace the ``step()`` method of *env* with a tracing function
     that calls *callbacks* with an events time, priority, ID and its
     instance just before it is processed.
     note: "event" here refers to simulaiton program events

     """
     def get_wrapper(env_step, callback):
         """Generate the wrapper for env.step()."""
         @wraps(env_step)
         def tracing_step():
             """Call *callback* for the next event if one exist before
             calling ``env.step()``."""
             if len(env._queue):
                 t, prio, eid, event = env._queue[0]
                 callback(t, prio, eid, event)
             return env_step()
         return tracing_step

     env.step = get_wrapper(env.step, callback)

def trace_monitor(data, t, prio, eid, event):
     data.append((t, eid, type(event)))

def test_process(env):
     yield env.timeout(1)


In [29]:
def event_log_append(env, caseid, time, activity, event_log):
    event_log.append((caseid, time, activity))
    yield env.timeout(0)

In [46]:
if obtain_reproducible_results:
    np.random.seed(9999)

#set up simulation trace monitoring
data = []
#bind data as first argument to monitor)
this_trace_monitor = partial(trace_monitor, data)

env = simpy.Environment()
trace(env, this_trace_monitor)

env.process(test_process(env))

#hold caseid values
caseid_queue1 = queue.Queue()
caseid_inv1 = queue.Queue()
caseid_queue2 = queue.Queue()
caseid_inv2 = queue.Queue()
caseid_transition = queue.Queue()

#setup limited stage 1 machines
available_machine1 = simpy.Resource(env, capacity = machine1)
available_machine2 = simpy.Resource(env, capacity = machine2)
caseid = -1

event_log = [(caseid,0,'null_start_simulation')]
env.process(event_log_append(env, caseid, env.now, 'start_simulation', event_log))

env.process(prep(env, caseid, caseid_queue1, caseid_inv1, event_log))

env.run(until = fixed_simulation_time)

Next prepared part for Stage 1:  312
Prepared part joins queue caseid = 1 time = 312 queue_length = 1
Next prepared part for Stage 1:  356
Begin_stage1_machining caseid = 1 time =  312 queue1_length = 1
Prepared part joins queue caseid = 2 time = 356 queue_length = 1
Next prepared part for Stage 1:  498
Begin_stage1_machining caseid = 2 time =  356 queue1_length = 1
Prepared part joins queue caseid = 3 time = 498 queue_length = 1
Next prepared part for Stage 1:  535
Begin_stage1_machining caseid = 3 time =  498 queue1_length = 1
Prepared part joins queue caseid = 4 time = 535 queue_length = 1
Next prepared part for Stage 1:  702
End_stage1_machining caseid= 1 time =  612 queue1_length = 1
Transition to stage 2 caseid= 1 time =  612 transition_queue_length = 1
Machined part joins queue2 caseid = 1 time = 612 queue_length = 1
Begin_stage2_machining caseid = 1 time =  612 queue2_length = 1
Begin_stage1_machining caseid = 4 time =  612 queue1_length = 1
End_stage1_machining caseid= 2 time 

End_stage1_machining caseid= 58 time =  10751 queue1_length = 15
Transition to stage 2 caseid= 58 time =  10751 transition_queue_length = 1
Machined part placed in Stage 2 Inventory. caseid= 58 time = 10751 inv_size = 5
Begin_stage1_machining caseid = 61 time =  10751 queue1_length = 15
Prepared part joins queue caseid = 76 time = 10764 queue_length = 15
Next prepared part for Stage 1:  10812
End_stage2_machining for inventory parts caseid= 52 time =  10773 queue2_length = 5
Begin_stage2_machining for inventory parts, caseid = 55 time =  10773 inv2_length = 5
Prepared part joins queue caseid = 77 time = 10812 queue_length = 16
Next prepared part for Stage 1:  10851
Prepared part joins queue caseid = 78 time = 10851 queue_length = 17
Next prepared part for Stage 1:  11020
End_stage1_machining caseid= 59 time =  10966 queue1_length = 17
Transition to stage 2 caseid= 59 time =  10966 transition_queue_length = 1
Machined part placed in Stage 2 Inventory. caseid= 59 time = 10966 inv_size = 

Transition to stage 2 caseid= 140 time =  22198 transition_queue_length = 1
Machined part placed in Stage 2 Inventory. caseid= 140 time = 22198 inv_size = 43
Begin_stage1_machining caseid = 119 time =  22198 queue1_length = 20
End_stage2_machining for inventory parts caseid= 76 time =  22222 queue2_length = 43
Begin_stage2_machining for inventory parts, caseid = 74 time =  22222 inv2_length = 43
Prepared part joins queue caseid = 141 time = 22278 queue_length = 20
Next prepared part for Stage 1:  22614
End_stage1_machining caseid= 118 time =  22300 queue1_length = 20
Transition to stage 2 caseid= 118 time =  22300 transition_queue_length = 1
Machined part placed in Stage 2 Inventory. caseid= 118 time = 22300 inv_size = 43
Begin_stage1_machining caseid = 120 time =  22300 queue1_length = 20
End_stage1_machining for inventory parts caseid= 139 time =  22372 queue1_length = 0
Transition to stage 2 caseid= 139 time =  22372 transition_queue_length = 1
Machined part placed in Stage 2 Invent

Prepared part placed in Stage 1 Inventory. caseid= 242 time = 37595 inv_size = 58
Next prepared part for Stage 1:  37637
Prepared part placed in Stage 1 Inventory. caseid= 243 time = 37637 inv_size = 59
Next prepared part for Stage 1:  37899
Prepared part placed in Stage 1 Inventory. caseid= 244 time = 37899 inv_size = 60
Next prepared part for Stage 1:  38069
End_stage2_machining for inventory parts caseid= 100 time =  38001 queue2_length = 62
Begin_stage2_machining for inventory parts, caseid = 103 time =  38001 inv2_length = 62
Prepared part placed in Stage 1 Inventory. caseid= 245 time = 38069 inv_size = 61
Next prepared part for Stage 1:  38240
End_stage2_machining for inventory parts caseid= 102 time =  38092 queue2_length = 61
Begin_stage2_machining for inventory parts, caseid = 104 time =  38092 inv2_length = 61
Prepared part placed in Stage 1 Inventory. caseid= 246 time = 38240 inv_size = 62
Next prepared part for Stage 1:  38296
Prepared part placed in Stage 1 Inventory. case

In [47]:
simulation_trace_file_name = 'simulation-program-trace-' + simulation_file_identifier + '.txt'
with open(simulation_trace_file_name, 'wt') as ftrace:
    for d in data:
        print(str(d), file = ftrace)
print()        
print('simulation program trace written to file:',simulation_trace_file_name)

# convert list of tuples to list of lists
event_log_list = [list(element) for element in event_log]

# convert to pandas data frame
caseid_list = []
time_list = []
activity_list = []
for d in event_log_list:
    if d[0] > 0:
        caseid_list.append(d[0])
        time_list.append(d[1])
        activity_list.append(d[2])
event_log_df = pd.DataFrame({'caseid':caseid_list,
                             'time':time_list,
                             'activity':activity_list})
print()
print('event log stored as Pandas data frame: event_log_df')

# save event log to comma-delimited text file
event_log_file_name = 'simulation-event-log-' + simulation_file_identifier + '.csv'
event_log_df.to_csv(event_log_file_name, index = False)  
print()
print('event log written to file:',event_log_file_name)


simulation program trace written to file: simulation-program-trace-1-days-3-5-10-20-3-20-2-10-20-30-30.txt

event log stored as Pandas data frame: event_log_df

event log written to file: simulation-event-log-1-days-3-5-10-20-3-20-2-10-20-30-30.csv


In [48]:
print(caseid_transition.qsize())

0


In [54]:
print()
print('Simulation parameter settings:')
print(machine1, 'Stage 1 machines')
print('  Stage 1 machine time settings (in minutes)')
print('    minimum:', min_machine1_time)
print('    mean:   ', mean_machine1_time)
print('    maximum:', max_machine1_time)
print(machine2, 'Stage 2 machines')
print('  Stage 2 machine time settings (in minutes)')
print('    minimum:', min_machine2_time)
print('    mean:   ', mean_machine2_time)
print('    maximum:', max_machine2_time)
print('mean observed during stages of machining times may deviate from the parameter setting')
print('due to censoring associated with the minimum and maximum')
print()
print('Parts set to be prepped over', mean_prep_time, 'minute(s) on average')
print('Parts will be stored in stage 1 inventory if there are', inventory1_queue_length, 'parts in the stage 1 queue')
print('Parts will be stored in stage 2 inventory if there are', inventory2_queue_length, 'parts in the stage 2 queue')
print('The simulation assumes the machine shop runs with two 8 hour shifts (16 hours of coverage per day)')
print('The simulation is set to run for ', simulation_days, ' days (',16*simulation_days,' hours /',16*60*simulation_days,' minutes)', sep ='')
print()
end_time = np.max(event_log_df["time"])
print('Results after ', end_time, ' seconds (', round(end_time/60, 2), ' minutes, ',round(end_time/(60*60),2),' hours):', sep = '')
caseid_list = pd.unique(event_log_df['caseid'])
print(len(caseid_list), 'unique parts prepped for stage 1')
print(len(event_log_df['activity'][event_log_df['activity']=='join_queue1']), 'prepped parts joined stage 1 queue')
print(len(event_log_df['activity'][event_log_df['activity']=='inv1']), 'prepped parts stored in stage 1 inventory')
print(len(event_log_df['activity'][event_log_df['activity']=='begin_stage1_machine']), 'prepped parts that began stage 1 machining process')
print(len(event_log_df['activity'][event_log_df['activity']=='end_stage1_machine']), 'prepped parts that finished stage 1 machining process')
print(caseid_queue1.qsize(), 'parts still in stage 1 queue')
print(caseid_inv1.qsize(), 'parts in stage 1 inventory')
print(len(event_log_df['activity'][event_log_df['activity']=='begin_stage2_machine']), 'machined parts that began stage 2 machining process')
print(len(event_log_df['activity'][event_log_df['activity']=='end_stage2_machine']), 'machined parts that finished stage 2 machining process')
print(caseid_queue2.qsize(), 'parts still in stage 2 queue')
print(caseid_inv2.qsize(), 'parts in stage 2 inventory')


Simulation parameter settings:
3 Stage 1 machines
  Stage 1 machine time settings (in minutes)
    minimum: 5
    mean:    10
    maximum: 20
2 Stage 2 machines
  Stage 2 machine time settings (in minutes)
    minimum: 10
    mean:    20
    maximum: 30
mean observed during stages of machining times may deviate from the parameter setting
due to censoring associated with the minimum and maximum

Parts set to be prepped over 3 minute(s) on average
Parts will be stored in stage 1 inventory if there are 20 parts in the stage 1 queue
Parts will be stored in stage 2 inventory if there are 30 parts in the stage 2 queue
The simulation assumes the machine shop runs with two 8 hour shifts (16 hours of coverage per day)
The simulation is set to run for 1 days (16 hours /960 minutes)

Results after 57554 seconds (959.23 minutes, 15.99 hours):
350 unique parts prepped for stage 1
164 prepped parts joined stage 1 queue
186 prepped parts stored in stage 1 inventory
164 prepped parts that began stage

In [52]:
print()
print('Simulation parameter settings:')
print(machine1, 'Stage 1 machines')
print('  Stage 1 machine time settings (in minutes)')
print('    minimum:', min_machine1_time)
print('    mean:   ', mean_machine1_time)
print('    maximum:', max_machine1_time)
print('mean observed stage 1 machine times may deviate from the parameter setting')
print('due to censoring associated with the minimum and maximum')
print()
print('Parts set to be prepped over', mean_prep_time, 'minute(s) on average')
print('Parts will be stored in stage 1 inventory if there are', inventory1_queue_length, 'parts in the stage 1 queue')
print('The simulation assumes the machine shop runs with two 8 hour shifts (16 hours of coverage per day)')
print('The simulation is set to run for ', simulation_days, ' days (',16*simulation_days,' hours /',16*60*simulation_days,' minutes)', sep ='')
print()
end_time = np.max(event_log_df["time"])
print('Results after ', end_time, ' seconds (', round(end_time/60, 2), ' minutes, ',round(end_time/(60*60),2),' hours):', sep = '')
caseid_list = pd.unique(event_log_df['caseid'])
print(len(caseid_list), 'unique parts prepped for stage 1')
print(len(event_log_df['activity'][event_log_df['activity']=='join_queue1']), 'prepped parts joined stage 1 queue')
print(len(event_log_df['activity'][event_log_df['activity']=='inv1']), 'prepped parts stored in stage 1 inventory')
print(len(event_log_df['activity'][event_log_df['activity']=='begin_stage1_machine']), 'prepped parts that began stage 1 machining process')
print(len(event_log_df['activity'][event_log_df['activity']=='end_stage1_machine']), 'prepped parts that finished stage 1 machining process')
print(caseid_queue1.qsize(), 'parts still in stage 1 queue')
print(caseid_inv1.qsize(), 'parts in stage 1 inventory')

# case-by-case logs are very useful for checking the logic of the simulation
case_by_case_event_file_name = 'simulation-program-case-by-case-events-' + simulation_file_identifier + '.txt'
with open(case_by_case_event_file_name, 'wt') as fcasedata:
    lastcase_prep_time = 0  # initialize for use with first case
    # create lists for storing time interval data 
    inter_prep_times = [] # computed across cases
    waiting_time = [] # computed within each case that has begun service
    machine1_time = [] # computed within each case that has ended service
    for thiscase in caseid_list:
        # select subset of rows for thiscase and use as a Pandas data frame
        thiscase_events = event_log_df[['caseid','time','activity']][event_log_df['caseid']==thiscase]
        print(file = fcasedata)
        print('events for caseid',thiscase, file = fcasedata)
        print(thiscase_events, file = fcasedata) 
        # compute inter-arrival times between cases
        thiscase_prep_time = thiscase_events.loc[thiscase_events['activity']=='prep_part', 'time'].values[0]
        inter_prep_time = thiscase_prep_time - lastcase_prep_time
        inter_prep_times.append(inter_prep_time)
        print(file = fcasedata)
        print('time between finished parts prepped (this case minus previous case):',inter_prep_time, 'seconds', file = fcasedata)
        lastcase_prep_time  = thiscase_prep_time # save for next case in the for-loop
        # compute waiting times within this case (must have begin_service event/activity)
        if thiscase_events.loc[thiscase_events['activity']=='begin_stage1_machine'].shape[0] == 1:
            thiscase_begin_machine1 = thiscase_events.loc[thiscase_events['activity']=='begin_stage1_machine', 'time'].values[0]
            thiscase_join_queue1 = thiscase_prep_time = thiscase_events.loc[thiscase_events['activity']=='join_queue1', 'time'].values[0]
            thiscase_waiting_time = thiscase_begin_machine1 - thiscase_join_queue1
            waiting_time.append(thiscase_waiting_time)
            print('waiting time for this case (time between joining queue and beginning stage 1 machining):',thiscase_waiting_time, 'seconds', file = fcasedata)
        # compute machine time within this case (must have end_service event/activity)
        if thiscase_events.loc[thiscase_events['activity']=='end_stage1_machine'].shape[0] == 1:
            thiscase_end_machine1 = thiscase_events.loc[thiscase_events['activity']=='end_stage1_machine', 'time'].values[0]
            thiscase_machine1_time = thiscase_end_machine1 - thiscase_begin_machine1
            machine1_time.append(thiscase_machine1_time)
            print('stage 1 machine time for this case (time between beginning and ending machining):',thiscase_machine1_time, 'seconds', file = fcasedata)

print()     
print('Summary statistics for part inter-prep times:')
print('  Minimum: ',round(np.min(inter_prep_times),2), ' seconds (' ,round(np.min(inter_prep_times)/60,2), ' minutes)',sep='')  
print('  Mean:    ',round(np.average(inter_prep_times),2), ' seconds (' ,round(np.average(inter_prep_times)/60,2), ' minutes)',sep='')  
print('  Maximum: ',round(np.max(inter_prep_times),2), ' seconds (' ,round(np.max(inter_prep_times)/60,2), ' minutes)',sep='')      
print()
print('Summary statistics for part wait times:')
print('  Minimum: ',round(np.min(waiting_time),2), ' seconds (' ,round(np.min(waiting_time)/60,2), ' minutes)',sep='')  
print('  Mean:    ',round(np.average(waiting_time),2), ' seconds (' ,round(np.average(waiting_time)/60,2), ' minutes)',sep='')  
print('  Maximum: ',round(np.max(waiting_time),2), ' seconds (' ,round(np.max(waiting_time)/60,2), ' minutes)',sep='')  
print()
print('Summary statistics for stage 1 machining times:')
print('  Minimum: ',round(np.min(machine1_time),2), ' seconds (' ,round(np.min(machine1_time)/60,2), ' minutes)',sep='')  
print('  Mean:    ',round(np.average(machine1_time),2), ' seconds (' ,round(np.average(machine1_time)/60,2), ' minutes)',sep='')  
print('  Maximum: ',round(np.max(machine1_time),2), ' seconds (' ,round(np.max(machine1_time)/60,2), ' minutes)',sep='')  

print()        
print('simulation case-by-case event data written to file:',case_by_case_event_file_name)
print("this log is most useful in checking the logic of the simulation")

# Define output_table data frame for use within KNIME
# KNIME is a low-code platform for data science
output_string_key = []
output_string_value = []

output_string_key.append("Minimum inter-prep_time")
output_string_value.append(str(round(np.min(inter_prep_times),2)))
output_string_key.append("Average inter-prep_time")
output_string_value.append(str(round(np.average(inter_prep_times),2)))
output_string_key.append("Maximum inter-prep_time")
output_string_value.append(str(round(np.max(inter_prep_times),2)))

output_string_key.append("Mimimum waiting time")
output_string_value.append(str(round(np.min(waiting_time),2)))
output_string_key.append("Average waiting time")
output_string_value.append(str(round(np.average(waiting_time),2)))
output_string_key.append("Maximum waiting time")
output_string_value.append(str(round(np.max(waiting_time),2)))

output_string_key.append("Mimimum stage 1 machine time")
output_string_value.append(str(round(np.min(machine1_time),2)))
output_string_key.append("Average stage 1 machine time")
output_string_value.append(str(round(np.average(machine1_time),2)))
output_string_key.append("Maximum stage 1 machine time")
output_string_value.append(str(round(np.max(machine1_time),2)))
#output_string_key.append("Stage 1 Inventory at end of simulation")
#output_string_value.append(str(round(np.max(caseid_inv1),2)))

output_table = pd.DataFrame({"key":output_string_key, "value":output_string_value})
output_table.set_index(['key'])



Simulation parameter settings:
3 Stage 1 machines
  Stage 1 machine time settings (in minutes)
    minimum: 5
    mean:    10
    maximum: 20
mean observed stage 1 machine times may deviate from the parameter setting
due to censoring associated with the minimum and maximum

Parts set to be prepped over 3 minute(s) on average
Parts will be stored in stage 1 inventory if there are 20 parts in the stage 1 queue
The simulation assumes the machine shop runs with two 8 hour shifts (16 hours of coverage per day)
The simulation is set to run for 1 days (16 hours /960 minutes)

Results after 57554 seconds (959.23 minutes, 15.99 hours):
350 unique parts prepped for stage 1
164 prepped parts joined stage 1 queue
186 prepped parts stored in stage 1 inventory
164 prepped parts that began stage 1 machining process
164 prepped parts that finished stage 1 machining process
20 parts still in stage 1 queue
166 parts in stage 1 inventory


IndexError: index 0 is out of bounds for axis 0 with size 0